# Instalações e Importações

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

# para renomear arquivos automaticamente

import os

# Opções

Aqui temos 2 opções pra baixar os dados, uma pro tabnet antigo e uma pro mais atual. Escolha a que você quer e rode a célula.

<div class='alert alert-info'>

-> Os dados das duas versões do tabnet são levemente diferentes (Mas são diferentes).
<br>    

-> A primeira, disponível em: http://tabnet.datasus.gov.br/cgi/tabcgi.exe?pni/cnv/cpniuf.def teve sua última atualização em 2019.
<br>

-> A segunda, disponível em: http://tabnet.datasus.gov.br/cgi/dhdat.exe?bd_pni/cpnibr.def teve sua última atualização em junho de 2021. Parece ser a atual.
<br>

->*No final do notebook há uma função pra renomear os arquivos de acordo com os anos.*
</div>

**CÓDIGO**

Semelhanças no código:

- Descomentar a linha *chrome_options.add_argument('--headless')* muda 2 coisas:
    1. Dá pra ver a mágica do navegador rodando e clicando sozinho, super divertido
    2. Os downloads irão pra pasta padrão de downloads, ao invés da pasta onde está rodando o notebook
    
Diferenças no código:

- O tabnet mais atual muda de endereço quando é feito o clique para gerar a tabela, então pediu ajustes específicos de código pra rodar. Ele também muda a ordem das linhas e colunas no momento da seleção, além da estrutura HTML (XPath).
- O tabnet antigo roda tudo na mesma aba e mesmo endereço.
- O código para a versão mais atual abre o navegador uma vez só e mantém a atuação trocando de abas. O código para a versão até 2019 fecha o navegador e abre outra instância a cada iteração do loop *for*.



## Tabnet Até 2019

In [2]:
# # confira o numero da linha e coluna do item desejado em http://tabnet.datasus.gov.br/cgi/tabcgi.exe?pni/cnv/cpniuf.def

linha: int = 3 # unidade da federacao

coluna: int = 4 # imuno

opcao: int = 2019 # Ano que sera feito o download

anos: int = int(input('Qtos anos deseja fazer o download: ')) # qtd de anos

link_por_ano: dict = {} # dicionario a ser preenchido para posterior renomeação automatica dos arquivos
    
for _ in range(anos): # numero de interações referente a qtd de anos
    # -------------- opções para abrir o site ------------ #
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)

    # -------------- Abrindo o site ------------ #
    driver.get("http://tabnet.datasus.gov.br/cgi/tabcgi.exe?pni/cnv/cpniuf.def")
    driver.maximize_window() 

    # -------------- localizando os elementos e clicando ------------ #
    linhas = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
      (By.XPATH, f'// *[ @ id = "L"] / option[{linha}]')))
    linhas.click()
    time.sleep(2)  

    colunas = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
      (By.XPATH, f'//*[@id="C"]/option[{coluna}]')))
    colunas.click()
    time.sleep(2)  

    conteudo_box = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
      (By.XPATH, '//select[@name="Incremento"]')))
    conteudo_box.click()
    time.sleep(2)  

    el_ano = driver.find_element(By.ID, 'A')
    select = Select(el_ano)
    select.deselect_all()
    select.select_by_visible_text('{}'.format(opcao))  

    # -------------- clicando no botão mostrar ------------ #
    mostra = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
      (By.XPATH, '/html/body/div/div/center/div/form/div[4]/div[2]/div[2]/input[1]')))
    mostra.click()
    time.sleep(5)  
    
    csv_link = driver.find_elements_by_xpath('/html/body/div/div/div[3]/table[1]/tbody/tr/td[1]/a')[0]
    link_por_ano[opcao] = csv_link.get_attribute('href')

    opcao -= 1 

    # -------------- clicando no botão download(csv) ------------ #
    download = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
      (By.XPATH, '/html/body/div/div/div[3]/table[1]/tbody/tr/td[1]/a')))
    download.click()

    # -------------- espera para fazer o download ------------ #
    time.sleep(5)  

    # -------------- fechando o navegador ------------ #
    driver.quit()

for ano, link in link_por_ano.items():
    print(ano, "-", link)

Qtos anos deseja fazer o download:  3


2019 - http://tabnet.datasus.gov.br/csv/A031721189_28_143_208.csv
2018 - http://tabnet.datasus.gov.br/csv/A031741189_28_143_208.csv
2017 - http://tabnet.datasus.gov.br/csv/A031801189_28_143_208.csv


## Tabnet Atual

In [4]:
# confira o numero da linha e coluna do item desejado em http://tabnet.datasus.gov.br/cgi/dhdat.exe?bd_pni/cpnibr.def

linha: int = 2 # unidade da federacao

coluna: int = 5 # imuno

opcao: int = 2021 # Ano que sera feito o download em cada iteração. Coloque o valor do ano mais recente desejado.

anos: int = int(input('Qtos anos deseja fazer o download: ')) # qtd de anos
    
## exemplo: opcao = 2020 e anos = 3 --> download dos anos 2020, 2019 e 2018
    
link_por_ano: dict = {} # dicionario a ser preenchido para posterior renomeação automatica dos arquivos
    
def datasus_tabnet_download(linha, coluna, opcao, anos, link_por_ano):    
#     -------------- opções para abrir o site ------------ #
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)

    # -------------- Abrindo o site ------------ #
    driver.get("http://tabnet.datasus.gov.br/cgi/dhdat.exe?bd_pni/cpnibr.def")
    driver.maximize_window() 
    
    for _ in range(anos): # numero de interações referente a qtd de anos

        # -------------- localizando os elementos e clicando ------------ #
        linhas = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
          (By.XPATH, f'// *[ @ id = "L"] / option[{linha}]'))) 
        linhas.click()
        time.sleep(2)  

        colunas = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
          (By.XPATH, f'//*[@id="C"]/option[{coluna}]'))) 
        colunas.click()
        time.sleep(2)  

        conteudo_box = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
          (By.XPATH, '//*[@name="Incremento"]'))) # cobertura vacinal
        conteudo_box.click()
        time.sleep(2)  

        el_ano = driver.find_element(By.ID, 'A')
        select = Select(el_ano)
        select.deselect_all()
        select.select_by_visible_text('{}'.format(opcao))  

        original_window = driver.current_window_handle

        # -------------- clicando no botão mostrar ------------ #
        mostra = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
          (By.XPATH, '/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]'))) 
        mostra.click()
        time.sleep(5)  

    #     wait = WebDriverWait(driver, 10)
    #     wait.until(EC.number_of_windows_to_be(2))

        # -------------- clicando no botão download(csv) ------------ #
        driver.switch_to_window(driver.window_handles[1])

        csv_link = driver.find_elements_by_xpath('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')[0]
        link_por_ano[opcao] = csv_link.get_attribute('href')

        opcao -= 1 

        download = WebDriverWait(driver, 30).until(EC.presence_of_element_located(
          (By.XPATH, '/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')))
        download.click()

        # -------------- espera para fazer o download ------------ #
        time.sleep(5)  
        
        
        #Close the tab or window
        driver.close()

        #Switch back to the old tab or window
        driver.switch_to.window(original_window)

    # -------------- fechando o navegador ------------ #
    driver.quit()
    
    
    
datasus_tabnet_download(linha, coluna, opcao, anos, link_por_ano)
    
for ano, link in link_por_ano.items():
    print(ano, "-", link)

Qtos anos deseja fazer o download:  6


<ipython-input-4-7d37533b5fab>:75: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])


2018 - http://tabnet.datasus.gov.br/cgi/csv/cpnibr16233908377.csv
2017 - http://tabnet.datasus.gov.br/cgi/csv/cpnibr16233908556.csv
2016 - http://tabnet.datasus.gov.br/cgi/csv/cpnibr16233908715.csv
2015 - http://tabnet.datasus.gov.br/cgi/csv/cpnibr16233908887.csv
2014 - http://tabnet.datasus.gov.br/cgi/csv/cpnibr16233909050.csv
2013 - http://tabnet.datasus.gov.br/cgi/csv/cpnibr16233909218.csv


# Renomeando arquivos automaticamente

In [4]:
# só testei em ubuntu + jupyter, então não sei se roda certinho em outros sistemas ou no collab. Talvez precise de pequenos ajustes

def renaming_downloaded_files(src: str, dst: str, base_rename: str):
    """src: source, caminho da pasta onde estão seus arquivos, provavelmente será a pasta Downloads
       dst: destino, caminho da pasta para onde voce quer mover o arquivo (caso não queira, é só usar a mesma string do src)
       base_rename: string comum à série toda. Será concatena ao ano. (Ex: coberturas_vacinais_por_imuno_segundo_unidade_da_federacao.csv"""
    
    for ano, link in link_por_ano.items():
        
        file = src + link.split("/")[-1]
        renamed_file = dst + str(ano) + '_' + base_rename
        if os.path.isfile(file):
            os.rename(file, renamed_file)
        else:
            print(file, 'does not exists')
        
    
renaming_downloaded_files(  src='',
                            dst='',
                            base_rename = 'coberturas_vacinais_por_imuno_segundo_unidade_da_federacao.csv')